In [1]:
ls

2016ElectionDataByState.csv
Cleaning Data - US President & Vice President.ipynb
General2016.csv
README.md
__pycache__/
config.py
proposal.txt
replies.py


In [16]:
import pandas as pd
import tweepy
import time
import json
# from replies import *
from pprint import pprint
from config import consumer_key, consumer_secret, access_token, access_token_secret
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [4]:
#import csv
file = "2016ElectionDataByState.csv"
df = pd.read_csv(file)
del df['Unnamed: 3']
df


,State,Clinton,Trump
0,Alabama,"729,547","1,318,255"
1,Alaska,"116,454","163,387"
2,Arizona,"1,161,167","1,252,401"
3,Arkansas,"380,494","684,872"
4,California,"8,753,792","4,483,814"
5,Colorado,"1,338,870","1,202,484"
6,Connecticut,"897,572","673,215"
7,Delaware,"235,603","185,127"
8,D. C.,"282,830","12,723"
9,Florida,"4,504,975","4,617,886"


In [25]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [5]:
# #show US Pres & Vice Pres only
# clean_df = df.loc[(df["Race"] == 'US President & Vice President')]
# clean_df.head()

In [6]:
# df = clean_df.groupby(["CounterType","Precinct","Party"])["SumOfCount"].sum().reset_index()  
# df.head()

In [7]:
# df.loc[(df["Party"].isin(['RPN','DPN']))].reset_index(drop=True)  

In [8]:
# Target Term
target_user = "@realDonaldTrump"

In [9]:
# Get all tweets from target user
tweet_list = []
public_tweets = api.user_timeline(target_user, page=1)
original_tweet = 0
# Loop through all tweets
for i, tweet in enumerate(public_tweets):
    tweet_list.append(tweet)
    replies_list = []
    orginal_tweet = tweet.get('id')

    replies = api.search(q='to:@realDonaldTrump',since_id=tweet.get('id'),count=100,geocode='41.8781,-87.6298,2000mi')
    while len(replies.get('statuses')) > 0 and replies.get('statuses')[0].get('id') > original_tweet:
        for reply in replies["statuses"]:
#             if reply.get('coordinates') != None:
            replies_list.append(reply) 
        replies = api.search(q='to:@realDonaldTrump',since_id=reply.get('id'), count=100,geocode='41.8781,-87.6298,2000mi')
        original_tweet = reply.get('id')  
        if (len(replies_list) > 4000):
            break
    break

    

In [10]:
len(tweet_list), len(replies_list)

(1, 4014)

In [11]:
pprint(tweet_list[0].get('id')), pprint(tweet_list[0].get('created_at')), pprint(tweet_list[0].get('text'))

1005479740000624640
'Sat Jun 09 16:00:25 +0000 2018'
('.@G7 Press Briefing in Charlevoix, Canada, prior to departing for Singapore! '
 'https://t.co/75y1SXuwuU')


(None, None, None)

In [12]:
# for reply in replies_list:
#     pprint(reply.get('id'))
#     pprint(reply.get('created_at'))
#     pprint(reply.get('text'))
#     pprint(reply.get('coordinates'))
#     break
    
replies_df = pd.DataFrame(replies_list)



In [13]:
replies_list[0].get('user').get('location')
replies_list[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jun 09 16:54:34 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/1…',
    'expanded_url': 'https://twitter.com/i/web/status/1005493366258356224',
    'indices': [106, 129],
    'url': 'https://t.co/jrhVbhAnIz'}],
  'user_mentions': [{'id': 25073877,
    'id_str': '25073877',
    'indices': [0, 16],
    'name': 'Donald J. Trump',
    'screen_name': 'realDonaldTrump'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1005493366258356224,
 'id_str': '1005493366258356224',
 'in_reply_to_screen_name': 'realDonaldTrump',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': 25073877,
 'in_reply_to_user_id_str': '25073877',
 'is_quote_status': True,
 'lang': 'en',
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'place': None,
 'possibly_sensitive': False,
 'quoted_status': {'contributors': No

In [26]:

flat_replies = []
# found_State = False
for reply in replies_list:
    if len(reply.get('user').get('location')) > 0:
        for state in states:
            if (reply.get('user').get('location').find(state) > 0):
                    break
        # Run Vader Analysis on each tweet
        results = analyzer.polarity_scores(reply.get('text'))
        reply_dict = {'id': reply.get('id'),
                    'text' : reply.get('text'),
                    'location': state,
                    'compound': results["compound"],
                    'pos': results["pos"],
                    'neu': results["neu"],
                    'neg': results["neg"]}
        flat_replies.append(reply_dict)

In [27]:
replies_df = pd.DataFrame(flat_replies)
replies_df

,compound,id,location,neg,neu,pos,text
0,-0.2263,1005493366258356224,RI,0.148,0.742,0.109,@realDonaldTrump a man who holds himself like...
1,0.0000,1005493328215883776,AZ,0.000,1.000,0.000,@realDonaldTrump https://t.co/nNe6Q7rBRr
2,0.8020,1005493307584270338,WY,0.000,0.714,0.286,@realDonaldTrump If you have the need to be a ...
3,-0.7650,1005493295395454976,WY,0.355,0.645,0.000,"@realDonaldTrump Bigly incompetent, not to men..."
4,0.0000,1005493283680907264,FL,0.000,1.000,0.000,@realDonaldTrump is going to grab Un’s pussy. ...
5,0.0000,1005493281520869377,VA,0.000,1.000,0.000,@realDonaldTrump https://t.co/JZZSK1nixa
6,0.8591,1005493262684229632,WY,0.000,0.627,0.373,@realDonaldTrump .Thank you for the news today...
7,0.7506,1005493260016447488,WY,0.000,0.795,0.205,"@realDonaldTrump And after you left, they are ..."
8,0.0000,1005493252647243779,WY,0.136,0.729,0.136,@realDonaldTrump @g7 THE FACT CHECKERS BRAINS ...
9,-0.4404,1005493250747195392,WY,0.153,0.847,0.000,"@realDonaldTrump looks small, week and outnumb..."


In [28]:
replies_df.groupby("location")["compound"].mean().reset_index()

,location,compound
0,AZ,-0.067933
1,CA,0.027785
2,DC,0.325200
3,FL,-0.290177
4,GA,-0.440400
5,IL,0.000000
6,IN,0.000000
7,NV,-0.359500
8,NY,-0.171163
9,OH,0.000000
